In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

from sklearn.preprocessing import MaxAbsScaler

# To remove the scientific notation from numpy arrays
np.set_printoptions(suppress=True)


2023-02-08 11:59:31.808618: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
#dataset = pd.read_pickle('SNR_BBH-injections-5000.pkl')
#dataset = pd.read_pickle('SNR_BBH-injections-reduced-100k.pkl')
dataset = pd.read_pickle('SNR_BBH-injections-reduced-1m.pkl')


dataset.head()

,ds,Mc,q,inc,spin1_mag,spin1_polar,spin1_azimuthal,spin2_mag,spin2_polar,spin2_azimuthal,coa_phase,dec,ra,pol,tc,SNR
0,1000.0,30.336864,2.002997,2.608599,0.559640,0.002940,1.788084,0.263034,0.911996,2.976439,0.0,-0.245609,3.745416,0.0,1.192530e+09,28.436068
1,1000.0,27.933059,1.063466,2.274400,0.070477,1.371662,4.080671,0.910459,0.400873,4.445331,0.0,0.252191,5.392380,0.0,1.192530e+09,9.574330
2,1000.0,19.412520,2.528205,1.988473,0.418485,1.392812,5.568183,0.264728,0.307003,2.490901,0.0,-1.131917,2.674948,0.0,1.192530e+09,11.382175
3,1000.0,21.674950,1.102297,1.628139,0.361641,0.843580,5.902105,0.606076,2.047297,1.159646,0.0,-1.227252,2.150521,0.0,1.192530e+09,5.773854
4,1000.0,38.872794,1.099620,0.809200,0.311437,0.246346,3.251720,0.399867,1.179204,3.741881,0.0,1.086481,3.558429,0.0,1.192530e+09,14.345895


In [3]:
dataset = dataset.loc[dataset["SNR"] <= 100]
dataset.shape


(999996, 16)

In [4]:
# Separate Target Variable and Predictor Variables
TargetVariable = ['SNR']
Predictors = ['Mc', 'q', 'inc', 'ra' , 'dec' , 'spin1_mag', 'spin1_polar', 'spin1_azimuthal', 'spin2_mag', 'spin2_polar', 'spin2_azimuthal']
#Predictors = ['ds', 'Mc', 'q', 'inc', 'ra' , 'dec' , 'spin1x', 'spin1y', 'spin1z', 'spin2x', 'spin2y', 'spin2z']

X = dataset[Predictors].values
y = dataset[TargetVariable].values
 
# Sandardization of data #
PredictorScaler = StandardScaler()
Xscaled = PredictorScaler.fit_transform(X)


TargetScaler =  StandardScaler()
yscaled = TargetScaler.fit_transform(y)

# Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xscaled, yscaled, test_size=0.3, random_state=42)
 
# Quick sanity check with the shapes of Training and testing datasets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(699997, 11)
(699997, 1)
(299999, 11)
(299999, 1)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.initializers import RandomNormal
from keras.layers import PReLU
from keras.initializers import Constant


# get input dimensions
input_dim = X_train.shape[1]

# number of neurons to start with
neurons = 50

# loop through different number of neurons
while neurons <= 500:
    
    # create ANN model
    model = Sequential()
    
    # set up initializer
    kernel_initializer = RandomNormal
    
    # add hidden layers with the current number of neurons
    model.add(Dense(units = neurons, input_dim=input_dim, kernel_initializer=kernel_initializer, bias_initializer='zeros'))
    model.add(PReLU(alpha_initializer=Constant(value=0.25)))          
    model.add(Dense(units = neurons, kernel_initializer=kernel_initializer, bias_initializer='zeros'))
    model.add(PReLU(alpha_initializer=Constant(value=0.25)))          
    model.add(Dense(units = neurons, kernel_initializer=kernel_initializer, bias_initializer='zeros'))
    model.add(PReLU(alpha_initializer=Constant(value=0.25)))
    model.add(Dense(units = neurons, kernel_initializer=kernel_initializer, bias_initializer='zeros'))
    model.add(PReLU(alpha_initializer=Constant(value=0.25)))          
    model.add(Dense(units = neurons, kernel_initializer=kernel_initializer, bias_initializer='zeros'))
    model.add(PReLU(alpha_initializer=Constant(value=0.25)))
    model.add(Dense(units = neurons, kernel_initializer=kernel_initializer, bias_initializer='zeros'))
    model.add(PReLU(alpha_initializer=Constant(value=0.25)))          
    
    # add the output layer
    model.add(Dense(1, kernel_initializer=kernel_initializer))
    
    # compile the model
    model.compile(loss='mean_squared_error', optimizer='Nadam')
    
    batch_size = 500 # 500 #50 # 500
    epochs = 800 #300 #100 # 300


    # Fitting the ANN to the Training set
    history = model.fit(X_train, y_train, validation_split=0.33, batch_size = batch_size, epochs = epochs, verbose=1)
    
    
    
    # Generating Predictions on testing data
    Predictions_scaled = model.predict(X_test)

# Scaling the test data back to original scale
    Test_Data = PredictorScaler.inverse_transform(X_test)

# Scaling the y_test Price data back to original price scale
    y_test_orig=TargetScaler.inverse_transform(y_test)

# Scaling the predicted Price data back to original price scale
    Predictions = TargetScaler.inverse_transform(Predictions_scaled)

# Create table with test data and predictions
    TestingData = pd.DataFrame( data = Test_Data, columns = Predictors)
    TestingData['SNR'] = y_test_orig
    TestingData['PredictedSNR'] = Predictions

# Computing the absolute percent error
    APE = 100*(abs( (TestingData['SNR']-TestingData['PredictedSNR'])/TestingData['SNR']) )
    TestingData['APE']=APE
    MAPE = np.mean(TestingData['APE'])
    Accuracy = 100 - MAPE

    # Computing the error
    Error = TestingData['SNR']-TestingData['PredictedSNR']
    TestingData['Error']=Error
    
    print("Number of neurons:", neurons, "Accuracy:", Accuracy)
    
    # increase the number of neurons
    neurons += 50


2023-02-08 11:59:32.579353: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-02-08 11:59:32.579886: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-02-08 11:59:32.603121: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 11:59:32.603222: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:2b:00.0 name: NVIDIA GeForce RTX 4080 computeCapability: 8.9
coreClock: 2.505GHz coreCount: 76 deviceMemorySize: 15.70GiB deviceMemoryBandwidth: 667.63GiB/s
2023-02-08 11:59:32.603235: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-02-08 11:59:32.603928: I tensorflow/stream_executor/platform/d

Epoch 1/800
  1/938 [..............................] - ETA: 11:58 - loss: 1.0293

2023-02-08 11:59:34.011888: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


938/938 [==============================] - 4s 4ms/step - loss: 0.3833 - val_loss: 0.1195
Epoch 2/800
938/938 [==============================] - 3s 3ms/step - loss: 0.1224 - val_loss: 0.1081
Epoch 3/800
938/938 [==============================] - 3s 3ms/step - loss: 0.1042 - val_loss: 0.0893
Epoch 4/800
938/938 [==============================] - 3s 4ms/step - loss: 0.0908 - val_loss: 0.0686
Epoch 5/800
938/938 [==============================] - 3s 3ms/step - loss: 0.0690 - val_loss: 0.0536
Epoch 6/800
938/938 [==============================] - 3s 3ms/step - loss: 0.0543 - val_loss: 0.0537
Epoch 7/800
938/938 [==============================] - 3s 3ms/step - loss: 0.0477 - val_loss: 0.0476
Epoch 8/800
938/938 [==============================] - 3s 4ms/step - loss: 0.0437 - val_loss: 0.0384
Epoch 9/800
938/938 [==============================] - 3s 4ms/step - loss: 0.0406 - val_loss: 0.0474
Epoch 10/800
938/938 [==============================] - 3s 4ms/step - loss: 0.0385 - val_loss: 0.0365
E